Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Инструкция по выполнению проекта

Загрузите и изучите данные.

Ответьте на вопрос и обоснуйте решение.
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
a. Изменится. Приведите примеры матриц.
b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

## Загрузка данных

In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from catboost import CatBoostClassifier
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import lightgbm as lgb

rnd_state = 12345
data  = pd.read_csv('/datasets/insurance.csv')

In [26]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [27]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [28]:
data.shape

(5000, 5)

In [29]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

## Умножение матриц

В этом задании вы можете записывать формулы в Jupyter Notebook.

Чтобы записать формулу внутри текста, окружите её символами доллара $; 
если снаружи — двойными символами \$$. Эти формулы записываются на языке вёрстки LaTeX.

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в LaTeX необязательно.

Обозначения:
    
X
 — матрица признаков (нулевой столбец состоит из единиц)
 
y
 — вектор целевого признака
 
P
 — матрица, на которую умножаются признаки
 
w
 — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:
    a = xw
    

Задача обучения:
w=argminMSE(Xw,y)

Формула обучения:
w = (X^TX)^(-1)X^(T)y

Ответ: Предсказания 
a
 не изменятся, качество линейной регрессии останется прежним

Обоснование:
    Для доказательства подставим вместо X - XP
    anew = XPwnew
    wnew = ((XP)^T XP)^(-1) (XP)^(T)y

подставим wnew в anew и преобразуем 
anew = XP((XP)^(T) XP)^(-1)(XP)^Ty = XP(P^TX^TXP)^(-1) P^(T)X^(T)y = XP(X^(T)XP)^(-1)(P^T)^(-1)P^(T)X^(T)y= XPP^(-1)(X^TX)^(-1)(P^T)^(-1)P^(T)X^(T)y=XE(X^TX)^(-1)EX^(T)y=X(X^TX)^(-1)X^(T)y=Xw

Пояснения:
    раскрыл транспортирование по свойству (AB)^T=B^T*A^T
    X^TX - квадратная обратимая матрица, P - по определению обратимая 
    (AB)^-1 = B^(-1)*A^(-1) 2 раза
    
    получаем что anew=a

##  Алгоритм преобразования

In [30]:
X = np.random.randn(4, 2)
P = np.random.randn(2, 2)

P_inv = np.linalg.inv(P)

X_ = X @ P
X

array([[-2.00163731, -0.37184254],
       [ 1.66902531, -0.43856974],
       [-0.53974145,  0.47698501],
       [ 3.24894392, -1.02122752]])

In [31]:
X_ @ P_inv

array([[-2.00163731, -0.37184254],
       [ 1.66902531, -0.43856974],
       [-0.53974145,  0.47698501],
       [ 3.24894392, -1.02122752]])

## Проверка алгоритма

In [32]:
class FeatPreparer(TransformerMixin, BaseEstimator):
    
    
    def fit(self, features, y=None, random_state=None):
        if random_state:
            np.random.seed(random_state)
            
        features_dim = features.shape[1]
        P = np.random.normal(size=(features_dim, features_dim))
        
        
        self.P = P
        self.features = features
        return self
        
    
    def transform(self, features, y=None):  
        return features.dot(self.P)

In [33]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=0)

In [34]:
features_new = FeatPreparer().fit_transform(features, random_state=rnd_state)
features.shape, features_new.shape

((5000, 4), (5000, 4))

In [35]:
linreg = LinearRegression()

linreg.fit(features, target)
predict1 = linreg.predict(features)

linreg.fit(features_new, target)
predict2 = linreg.predict(features_new)

In [36]:
print(r2_score(target, predict1))

0.4249455028666801


In [37]:
print(r2_score(target, predict2))

0.42494550286667687


In [38]:
np.isclose(r2_score(target, predict1), r2_score(target, predict2))

True

Метрика R2 осталась без особых изменений, даже после преобразования. Следовательно, при умножении признаков на обратимую матрицу не повлияет на качество линейной регрессию, как ранее и доказал. Моё доказательство оказалось верным.